- smoking.csv와 competition_format에 있는 데이터에 전처리를 모두 적용하고자 전처리 코드 함수화 후 재업로드합니다.
- 추가하거나 수정하실 사항 있으시면 자유롭게 첨삭해주시면 감사하겠습니다.
- bmi 컬럼 추가를 위해 계산식을 포함시키고 주석처리 해뒀습니다 - 22/9/27
- wwi 컬럼 추가를 위해 계산식을 포함시키고 주석처리 해뒀습니다 - 22/9/28
    - 허리둘레를 체중의 제곱근으로 나눈 값
    - "waist(cm)"/sqrt("weight(kg)")
    - WWI 수치가 높으면 지방이 많은 비만 환자이고 수치가 낮으면 근육이 상대적으로 많은 정상 체중이라는 의미
- hearing(left), hearing(right) 피처 1, 2 => 1, 0 으로 변환 -22/9/28

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder # for encoding
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #for standardization
from sklearn.model_selection import train_test_split

In [ ]:
def preprocessing(df_scaling, scaled_form = 'MinMaxScaler()'):
    # 불필요한 컬럼 제거
    if 'ID' in df_scaling:
        df_scaling = df_scaling.drop("ID", axis = 1)
        if len(df_scaling.columns) == 1:
            return df_scaling
        
        df_scaling = df_scaling.drop('oral', axis = 1) 
        # 범주형 피처 레이블 인코딩 
        cate_features = df_scaling[['gender','tartar']]

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["gender"])
        df_scaling["gender"] = lbe.fit_transform(df_scaling["gender"])

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["tartar"])
        df_scaling["tartar"] = lbe.fit_transform(df_scaling["tartar"])
        # hearing 피처 1, 2 => 1, 0으로 변환
        df_scaling['hearing(left)'] = df_scaling['hearing(left)'].apply(lambda x: x-2 if x ==2.0 else x )
        df_scaling['hearing(right)'] = df_scaling['hearing(right)'].apply(lambda x: x-2 if x ==2.0 else x )
        
#         # BMI 지수 계산
#         # bmi = kg/m^2
#         df_scaling['bmi'] = df_scaling['weight(kg)']/((df_scaling['height(cm)']*0.01)**2)
#         # wwi(비만 지수) 지수 계산
#         df_scaling['wwi'] = df_scaling['waist(cm)']/(df_scaling['weight(kg)'].apply(np.sqrt))

        # 정규화 작업
        cate_features = df_scaling[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
        scaled_features = df_scaling.drop(cate_features.columns, axis=1)
        if scaled_form == 'StandardScaler()':
            # Standard scaler
            scaler = StandardScaler()
            scaler.fit(scaled_features)
            scaled = scaler.transform(scaled_features)
            standard_scaled = pd.DataFrame(scaled,columns = scaled_features.columns)
            standard_scaled[cate_features.columns] = cate_features

            return standard_scaled
        else:
            # MinMax scaler
            scaler_M = MinMaxScaler()
            scaler_M.fit(scaled_features)
            scaled_M = scaler_M.transform(scaled_features)
            min_max_scaled = pd.DataFrame(scaled_M,columns = scaled_features.columns)
            min_max_scaled[cate_features.columns] = cate_features

            return min_max_scaled

In [ ]:
smoking = pd.read_csv('./data/Smoking_raw/smoking.csv')

In [ ]:
scaled_data = preprocessing(smoking)

In [ ]:
target = scaled_data['smoking']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_data, target, test_size = 0.1, random_state=0)

# competition_format

In [ ]:
# competition_format
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')

In [ ]:
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)
y_train = preprocessing(y_train)
y_test = preprocessing(y_test)

In [ ]:
x_test

In [ ]:
y_train

In [ ]:
y_test